In [1]:
import os
import pandas
import numpy as np
import rpy2.robjects
readRDS = rpy2.robjects.r['readRDS']
saveRDS= rpy2.robjects.r["saveRDS"]
from rpy2.robjects import pandas2ri
pandas2ri.activate()


r_code_get_rowandcolnames = '''
get_both <- function(ref_names = "reference_fruits.rds", mat = NULL) {
  ref_names <- readRDS(ref_names)
  if (!is.null(mat)) {
    return(list(  rownames(ref_names[[mat]]), colnames(ref_names[[mat]]) ))
  } else {
    return(list(rownames(ref_names),colnames(ref_names) ))
  }
}
'''
rpy2.robjects.r(r_code_get_rowandcolnames)
get_both_row_col = rpy2.robjects.r['get_both']


# # Function to convert R object to pandas DataFrame or numpy array
def r_object_to_python(r_object,file,element_name):
    try:
        # Try to convert to pandas DataFrame
        return pandas2ri.rpy2py(r_object)
    except NotImplementedError:
        rows, columns =get_both_row_col(file,element_name)
        if(isinstance(columns, type (rpy2.robjects.NULL))):
            df = pandas.DataFrame(r_object, index=rows)
        else: 
            columns = list(columns)
            df = pandas.DataFrame(r_object, columns=columns, index=rows)
        return df

# Function to extract named data elements and convert appropriately
def extract_data_element(data, file, element_name):
    if element_name in data.names:
        element = data.rx2(element_name)
        return r_object_to_python(element,file,element_name)
    return None

dir_name = "/Users/guillaume/Documents/HADACA/starting_kit_phase2-3/data"+os.sep

datasets_list = [filename for filename in os.listdir(dir_name) if filename.startswith("mixes")]

ref_file = os.path.join(dir_name, "reference_pdac.rds")
print("reading reference file")
reference_data = readRDS(ref_file)
ref_bulkRNA = extract_data_element(reference_data,ref_file, 'ref_bulkRNA') 
ref_met = extract_data_element(reference_data,ref_file, 'ref_met')

reading reference file


In [2]:
illumina_meth = pandas.read_csv("/Users/guillaume/Documents/HADACA/starting_kit_phase2-3/more_data/illumina_meth.csv", index_col=0)
inter_probes = list(np.intersect1d(illumina_meth.index, ref_met.index))
illumina_meth_ref = illumina_meth.loc[inter_probes]
exploded_illumina_ref = illumina_meth_ref["UCSC_RefGene_Name"].str.split(";").to_frame().explode("UCSC_RefGene_Name")
exploded_illumina_ref = exploded_illumina_ref.dropna()
common_bulk_meth = np.intersect1d(exploded_illumina_ref["UCSC_RefGene_Name"], ref_bulkRNA.index)
exploded_illumina_ref_rna = exploded_illumina_ref[exploded_illumina_ref["UCSC_RefGene_Name"].isin(common_bulk_meth)]
exploded_illumina_ref_rna = exploded_illumina_ref_rna.drop_duplicates()
exploded_illumina_ref_rna.to_csv("/Users/guillaume/Documents/HADACA/starting_kit_phase2-3/attachement/mapping_meth_rna.csv")
exploded_illumina_ref_rna


,UCSC_RefGene_Name
cg00002426,SLMAP
cg00003994,MEOX2
cg00006414,ZNF425
cg00006414,ZNF398
cg00007981,PANX1
...,...
cg27610250,SLC25A46
cg27611781,PDXP
cg27625732,TBC1D13
cg27638713,MCM3
